In [ ]:
from os import getenv
from pathlib import Path

input_folder = Path(getenv(
    'CROSSCOMPUTE_INPUT_FOLDER', 'batches/djibouti/input'))
output_folder = Path(getenv(
    'CROSSCOMPUTE_OUTPUT_FOLDER', 'batches/djibouti/output'))
log_folder = Path(getenv(
    'CROSSCOMPUTE_LOG_FOLDER', 'batches/djibouti/log'))
debug_folder = Path(getenv(
    'CROSSCOMPUTE_DEBUG_FOLDER', 'batches/djibouti/debug'))

In [ ]:
from os.path import splitext
from pandas import ExcelWriter, read_csv

specifications_path = debug_folder / 'specifications.xlsx'
excel_writer = ExcelWriter(specifications_path)
for file_name in [
    'scenario-info.csv',
    'scenario-parameters.csv',
    'specs-data.csv',
]:
    table = read_csv(input_folder / file_name)
    sheet_name = ''.join(_.title() for _ in splitext(file_name)[0].split('-'))
    table.to_excel(excel_writer, sheet_name=sheet_name, index=False)
excel_writer.close()

In [ ]:
from onsset.runner import calibration, scenario

settlements_path = input_folder / 'settlements.csv'
calibrated_specifications_path = debug_folder / 'specifications.xlsx'
calibrated_settlements_path = debug_folder / 'settlements.csv'
calibration(
    specs_path=specifications_path,
    csv_path=settlements_path,
    specs_path_calib=calibrated_specifications_path,
    calibrated_csv_path=calibrated_settlements_path)
scenario(
    specs_path=calibrated_specifications_path,
    calibrated_csv_path=calibrated_settlements_path,
    results_folder=log_folder,
    summary_folder=log_folder)

In [ ]:
from os import rename

summary_path = log_folder / 'summary.csv'
rename(next(log_folder.glob('*-1-*_summary.csv')), summary_path)
summary_table = read_csv(summary_path)

results_path = log_folder / 'results.csv'
rename(next(log_folder.glob('*-1-*.csv')), results_path)
results_table = read_csv(results_path)

In [ ]:
specs_data_table = read_csv(input_folder / 'specs-data.csv')
country_name = specs_data_table['Country'][0]
country_name

In [ ]:
target_year = summary_table.columns[-1]

In [ ]:
t = summary_table[['Unnamed: 0', target_year]]
t = t.set_index('Unnamed: 0')
t = t.squeeze()
t = t[t != 0]
t

In [ ]:
population_keys = [
    _ for _ in t.keys() if _.startswith('1.Population')]
population_keys

In [ ]:
technology_codes = [
    _.replace('1.Population_', '') for _ in population_keys]
technology_codes

In [ ]:
technology_name_by_code = {
    'Grid': 'Grid Extension',
    'SA_PV': 'Stand Alone - Photovoltaic',
    'MG_PV': 'Mini Grid - Photovoltaic',
    'MG_Wind': 'Mini Grid - Wind',
}
technology_names = [technology_name_by_code[
    _] for _ in technology_codes]
technology_names

In [ ]:
from math import ceil
from pandas import DataFrame

def get_values(prefix):
    return t[t.index.str.startswith(prefix)].reset_index(drop=True).apply(lambda _: '{:,d}'.format(ceil(_)))

overview_table = DataFrame()
overview_table['Electrification Technology'] = technology_names
overview_table['Population Connected'] = get_values('1.Population')
overview_table['Investment Required'] = get_values('4.Investment')
overview_table['Capacity Added'] = get_values('3.Capacity')
overview_table

In [ ]:
import json

with (output_folder / 'variables.dictionary').open('wt') as f:
    json.dump({
        'country_name': country_name,
        'target_year': target_year,
    }, f)

In [ ]:
with (output_folder / 'map.geojson').open('wt') as f:
    json.dump({
        'type': 'FeatureCollection',
        'features': [{
            'type': 'Feature',
            'geometry': {
                'type': 'Point',
                'coordinates': [_['X_deg'], _['Y_deg']],
            },
        } for i, _ in results_table.iterrows()],
    }, f)

In [ ]:
overview_table.to_json(
    output_folder / 'overview.json',
    orient='split',
    index=False)